In [1]:
import pandas as pd
import numpy as np
import random
import copy
import unittest

#### Определим "популярность" товара, как его проданный совокупный объем в штуках. С таким определением таблица orders является излишней для выполнения поставленной задачи. 

In [2]:
def BuildReport(orderLines,numproducts=10):
    if type(orderLines)!=type(pd.DataFrame()):
        raise TypeError
    

    if list(orderLines.columns)!=["OrderId","ProductId","Price"]:
        raise Exception("InputError")
    
    if not orderLines.empty:
        # numproducts - число наиболее популярных продуктов, которое необходимо представить в отчете

        data = copy.deepcopy(orderLines)

        # Рассчитаем сумму каждого заказа:
        Order_Revenue=data.groupby(["OrderId"])[["Price"]].sum().reset_index()
        Order_Revenue.columns=["OrderId","Order_Revenue"]
        data=data.merge(Order_Revenue,on=["OrderId"])

        # Рассчитаем средний чек. Для каждого товара определим среднюю сумму заказа, в котором он присутствовал.
        # Первый groupby необходим для того, чтобы правильно учитывать заказы, 
        # в которых один и тот же товар присутствовал несколько раз:
        Average_Bill=data.groupby(["ProductId","OrderId"])[["Order_Revenue"]].max().reset_index()\
        .groupby("ProductId")[["Order_Revenue"]].mean().reset_index()
        Average_Bill.columns=["ProductId","Average_Bill"]
        data=data.merge(Average_Bill,on=["ProductId"])

        # Выберем первые numproducts товаров для формирования отчета. Product_count - объем продажи данного товара (шт.),
        # Total_Revenue - объем продажи данного товара (руб.), Average_Bill - средний чек.
        res=data.groupby("ProductId").agg({"ProductId":["count"],"Price":["sum"],"Average_Bill":["max"]}).reset_index()
        res.columns=["ProductId","Product_count","Total_Revenue","Average_Bill"]
        res.sort_values(by=["Product_count"],ascending=False,inplace=True)

        return res.iloc[:numproducts,:]
    else:
        return pd.DataFrame(columns=["ProductId","Product_count","Total_Revenue","Average_Bill"])


In [3]:
class Test(unittest.TestCase):

    def test_type_mismatch(self):
        with self.assertRaises(TypeError):
            BuildReport(7)
    
    def test_input_mismatch(self):
        temp = pd.DataFrame(columns=["ProductId","Price"])
        with self.assertRaises(Exception):
            BuildReport(temp)
            
    def test_empty(self):
        temp = pd.DataFrame(columns=["OrderId","ProductId","Price"])
        res=BuildReport(temp).to_dict()
        target=pd.DataFrame(columns=["ProductId","Product_count","Total_Revenue","Average_Bill"]).to_dict()
        self.assertEqual(res,target)
        
    def test_single(self):
        temp = pd.DataFrame(data=np.array([1,1,1]).reshape((1,3)) ,columns=["OrderId","ProductId","Price"])
        res=BuildReport(temp).to_dict()
        target=pd.DataFrame(data=np.array([1,1,1,1]).reshape((1,4)) 
        ,columns=["ProductId","Product_count","Total_Revenue","Average_Bill"]).to_dict()
        self.assertEqual(res,target)

    def test_normal(self):
        
        orders=[x for x in range(1,1001)]
        products=[x for x in range(1,1001)]
        prices=[10 for x in range(1,1001)]
        
        temp = pd.DataFrame(data=np.array([orders,products,prices]).T ,columns=["OrderId","ProductId","Price"])
        
        res=BuildReport(temp).to_dict()
        
        prod_counts=[1 for x in range(1,1001)]
        revenues = [10 for x in range(1,1001)]
        bills=[10 for x in range(1,1001)]
        
        res=BuildReport(temp,1000).to_dict()
        target=pd.DataFrame(data=np.array([products,prod_counts,revenues,bills]).T 
        ,columns=["ProductId","Product_count","Total_Revenue","Average_Bill"]).to_dict()
        self.assertEqual(res,target)